# AWS Inferentia inference on Amazon EC2 Inf1 instance
This example demonstrates AWS Inferentia inference with TensorFlow and AWS Neuron SDK compiler and runtime

This example was tested on Amazon EC2 `inf1.xlarge` the following AWS Deep Learning AMI: 
`Deep Learning AMI (Ubuntu 18.04) Version 35.0`

Run this notebook using the following conda environment:
`aws_neuron_tensorflow_p36`

Prepare your imagenet validation TFRecord files using the following helper script:
https://github.com/tensorflow/models/blob/archive/research/inception/inception/data/download_and_preprocess_imagenet.sh

Save it to `/home/ubuntu/datasets/` or update the dataset location in the `get_dataset()` function

In [1]:
# !pip install matplotlib pandas

In [2]:
# !/opt/aws/neuron/bin/neuron-cli reset
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import time
import shutil
import json
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.neuron as tfn
import tensorflow.compat.v1.keras as keras
# from tensorflow.keras.applications import ( 
#     xception,
#     vgg16,
#     vgg19,
#     resnet,
#     resnet50,
#     resnet_v2,
#     inception_v3,
#     inception_resnet_v2,
#     mobilenet,
#     densenet,
#     nasnet,
#     mobilenet_v2
# )
# from tensorflow.keras.preprocessing import image
# from concurrent import futures
# from itertools import compress

# models = {
#     'xception':xception,
#     'vgg16':vgg16,
#     'vgg19':vgg19,
#     'resnet50':resnet50,
#     'resnet101':resnet,
#     'resnet152':resnet,
#     'resnet50_v2':resnet_v2,
#     'resnet101_v2':resnet_v2,
#     'resnet152_v2':resnet_v2,
# #     'resnext50':resnext,
# #     'resnext101':resnext,
#     'inception_v3':inception_v3,
#     'inception_resnet_v2':inception_resnet_v2,
#     'mobilenet':mobilenet,
#     'densenet121':densenet,
#     'densenet169':densenet,
#     'densenet201':densenet,
#     'nasnetlarge':nasnet,
#     'nasnetmobile':nasnet,
#     'mobilenet_v2':mobilenet_v2
# }

models_detail = {
#     'xception':xception.Xception(weights='imagenet',include_top=False),
#     'vgg16':vgg16.VGG16(weights='imagenet'),
#     'vgg19':vgg19.VGG19(weights='imagenet'),
#     'resnet50':resnet.ResNet50(weights='imagenet'),
#     'resnet101':resnet.ResNet101(weights='imagenet'),
#     'resnet152':resnet.ResNet152(weights='imagenet'),
#     'resnet50_v2':resnet_v2.ResNet50V2(weights='imagenet'),
#     'resnet101_v2':resnet_v2.ResNet101V2(weights='imagenet'),
#     'resnet152_v2':resnet_v2.ResNet152V2(weights='imagenet'),
#     'resnext50':resnext.ResNeXt50(weights='imagenet'),
#     'resnext101':resnext.ResNeXt101(weights='imagenet'),
#     'inception_v3':inception_v3.InceptionV3(weights='imagenet',include_top=False),
#     'inception_resnet_v2':inception_resnet_v2.InceptionResNetV2(weights='imagenet'),
#     'mobilenet':mobilenet.MobileNet(weights='imagenet'),
#     'densenet121':densenet.DenseNet121(weights='imagenet'),
#     'densenet169':densenet.DenseNet169(weights='imagenet'),
#     'densenet201':densenet.DenseNet201(weights='imagenet'),
#     'nasnetlarge':nasnet.NASNetLarge(weights='imagenet'),
#     'nasnetmobile':nasnet.NASNetMobile(weights='imagenet'),
#     'mobilenet_v2':mobilenet_v2.MobileNetV2(weights='imagenet')
}

print('test')

2022-11-17 04:58:17.105267: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-17 04:58:17.105303: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-52-177.us-west-2.compute.internal): /proc/driver/nvidia/version does not exist


test


### Resnet50 FP32 saved model

In [3]:
# # Export SavedModel
# model_type = 'resnet50'

# saved_model_dir = f'{model_type}_saved_model'
# shutil.rmtree(saved_model_dir, ignore_errors=True)

keras.backend.set_learning_phase(0)
# model = ResNet50(weights='imagenet')
# tf.saved_model.simple_save(session = keras.backend.get_session(),
#                            export_dir = saved_model_dir,
#                            inputs = {'input_1:0': model.inputs[0]},
#                            outputs = {'probs/Softmax:0': model.outputs[0]})

/home/ec2-user/cloud-hw-inference/Inferentia/tensorflow_venv/lib64/python3.7/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


### Use `tf.data` to read ImageNet validation dataset

In [4]:
def deserialize_image_record(record):
    feature_map = {'image/encoded': tf.io.FixedLenFeature([], tf.string, ''),
                  'image/class/label': tf.io.FixedLenFeature([1], tf.int64, -1)}
    obj = tf.io.parse_single_example(serialized=record, features=feature_map)
    imgdata = obj['image/encoded']
    label = tf.cast(obj['image/class/label'], tf.int32)   
    return imgdata, label

def val_preprocessing(record):
    imgdata, label = deserialize_image_record(record)
    label -= 1
    image = tf.io.decode_jpeg(imgdata, channels=3, 
                              fancy_upscaling=False, 
                              dct_method='INTEGER_FAST')

    shape = tf.shape(image)
    height = tf.cast(shape[0], tf.float32)
    width = tf.cast(shape[1], tf.float32)
    side = tf.cast(tf.convert_to_tensor(256, dtype=tf.int32), tf.float32)
    
    scale = tf.cond(tf.greater(height, width),
                  lambda: side / width,
                  lambda: side / height)
    
    new_height = tf.cast(tf.math.rint(height * scale), tf.int32)
    new_width = tf.cast(tf.math.rint(width * scale), tf.int32)
    
    image = tf.image.resize(image, [new_height, new_width], method='bicubic')
    image = tf.image.resize_with_crop_or_pad(image, 224, 224)
    
#     image = models[model_type].preprocess_input(image)
    
    return image, label

def get_dataset(batch_size, use_cache=False):
    data_dir = '/home/ec2-user/datasets/images-50000/*'
    files = tf.io.gfile.glob(os.path.join(data_dir))
    dataset = tf.data.TFRecordDataset(files)
    
    dataset = dataset.map(map_func=val_preprocessing, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat(count=1)
    
    if use_cache:
        shutil.rmtree('tfdatacache', ignore_errors=True)
        os.mkdir('tfdatacache')
        dataset = dataset.cache(f'./tfdatacache/imagenet_val')
    
    return dataset

## Single AWS Inferentia chip execution
* Single core compiled models with automatic data parallel model upto 4 cores
* Multi-core compiled models for pipeline execution

In [30]:
import os
    
def inf1_predict_benchmark_single_threaded(neuron_saved_model_name, batch_size, user_batch_size, num_cores, use_cache=False, warm_up=10):
    print(f'Running model {neuron_saved_model_name}, user_batch_size: {user_batch_size}\n')

    model_inf1 = tf.saved_model.load(neuron_saved_model_name)
    
    inference_function = model_inf1
    walltime_start = time.time()
    first_iter_time = 0
    iter_times = []
    pred_labels = []
    actual_labels = []
    total_datas = 50000
    display_every = 1000
    display_threshold = display_every
    
    ds = get_dataset(user_batch_size, use_cache)
    
    image_size=14
    input_channel = 512
    batch_size = 1
    input_shape = (batch_size, image_size, image_size, input_channel)
    data_array = np.random.uniform(0, 255, size=input_shape).astype("float32")
    ydata_array = np.random.uniform(0, 255, size=(1,1000)).astype("float32")
    
    data_array = tf.convert_to_tensor(data_array, dtype=tf.float32)
    
    load_start = time.time()
    load_time = time.time() - load_start
    counter = 0
    print(model_inf1)
#     for batch, batch_labels in ds:
#         start_time = time.time()
#         yhat_np = inference_function(batch)
#         if counter ==0:
#             first_iter_time = time.time() - start_time
#         else:
#             iter_times.append(time.time() - start_time)
            
#         actual_labels.extend(label for label_list in batch_labels for label in label_list)
#         pred_labels.extend(list(np.argmax(yhat_np, axis=1)))

#         if counter*batch_size >= display_threshold:
#             print(f'Images {counter*batch_size}/{total_datas}. Average i/s {np.mean(batch_size/np.array(iter_times[-display_every:]))}')
#             display_threshold+=display_every

#         counter+=1
#         if counter == 10:
#             break
    for i in range(10):
        start_time = time.time()
        inference_function(data_array)
        iter_times.append(time.time() - start_time)
    
    print(iter_times)
    iter_times = np.array(iter_times)
    acc_inf1 = np.sum(np.array(actual_labels) == np.array(pred_labels))/len(actual_labels)
    results = pd.DataFrame(columns = [f'inf1_tf2_{model_type}_{batch_size}'])
    results.loc['batch_size']              = [batch_size]
#     results.loc['accuracy']                = [acc_inf1]
    results.loc['first_prediction_time']   = [first_iter_time * 1000]
    results.loc['average_prediction_time'] = [np.median(iter_times) * 1000]
    results.loc['load_time']               = [load_time]
    results.loc['wall_time']               = [time.time() - walltime_start]

    return results, iter_times

In [31]:
model_type = 'vgg16_conv512_14'
# https://github.com/tensorflow/tensorflow/issues/29931
# temp = tf.zeros([8, 224, 224, 3])
# _ = models[model_type].preprocess_input(temp)

# testing batch size
batch_list = [1]
batch_size = 1
num_cores = 1
user_batch = 1
num_of_cores = [1]
user_batchs = [1]
inf1_compiled_model_dir = f'models/{model_type}_inf1_saved_models'

for user_batch in user_batchs:
    iter_ds = pd.DataFrame()
    results = pd.DataFrame()

    opt ={'batch_size': batch_size, 'num_cores': num_of_cores}
    col_name = lambda opt: f'inf1_{batch_size}_multicores_{num_cores}'
    res, iter_times = inf1_predict_benchmark_single_threaded(inf1_compiled_model_dir,
                                                                             batch_size = batch_size,
                                                                             user_batch_size = batch_size*user_batch,
                                                                             num_cores = num_cores,
                                                                             use_cache=False, 
                                                                             warm_up=10)

    iter_ds = pd.concat([iter_ds, pd.DataFrame(iter_times, columns=[col_name(opt)])], axis=1)
    results = pd.concat([results, res], axis=1)
display(results)

Running model models/vgg16_conv512_14_inf1_saved_models, user_batch_size: 1

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f6437918290>
[0.11273360252380371, 0.001402139663696289, 0.0012156963348388672, 0.0011475086212158203, 0.0008962154388427734, 0.0009481906890869141, 0.0009105205535888672, 0.0009219646453857422, 0.0008511543273925781, 0.0008783340454101562]


/home/ec2-user/cloud-hw-inference/Inferentia/tensorflow_venv/lib64/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in long_scalars


,inf1_tf2_vgg16_conv512_14_1
batch_size,1
first_prediction_time,0
average_prediction_time,0.935078
load_time,0.0
wall_time,0.239481
